In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD, RMSprop, Adam
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.callbacks import TensorBoard
from keras import backend as K

import numpy as np

Using TensorFlow backend.


# TODO
- more aggresive data augmentation
- more aggressive dropout
- use of L1 and L2 regularization (also known as "weight decay")
- fine-tuning one more convolutional block (alongside greater regularization)


In [2]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data/train'
validation_data_dir = 'data/valid'
test_data_dir = 'data/test'
nb_train_samples = 6363
nb_epochs = 15
batch_size = 16
nb_classes = 3

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
itr = train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=nb_train_samples,
    class_mode='categorical')

Found 6363 images belonging to 3 classes.


In [5]:
X, y = itr.next()

In [6]:
print(X.shape)
print(y.shape)

(6363, 224, 224, 3)
(6363, 3)


In [7]:
pre_processed_X = preprocess_input(X)

In [8]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')

Model loaded.


In [9]:
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)

In [10]:
# this is the model we will train
# add your top layer block to your base model
model = Model(base_model.input, predictions)

In [11]:
def printAllLayers():
    for i, layer in enumerate(model.layers):
       print(i, layer.name)

In [15]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
printAllLayers()

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [16]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:19]:
    layer.trainable = False

In [17]:
def printTrainableLayers():
    for i, layer in enumerate(model.layers):
       if layer.trainable == True: 
           print(i, layer.name)

In [18]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
printTrainableLayers()

19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [19]:
# checkpoint
filepath="vgg16-top-weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [20]:
#histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list.append(tbCallBack)

In [21]:
#opt = RMSprop(lr=0.1)
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08 )# decay=0.0
#SGD(lr=1e-4, momentum=0.9)
#opt=SGD(lr=0.01, momentum=0.9, nesterov=True, decay=0.5)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Goal: < 0.96188 val_loss

In [24]:
history = model.fit(pre_processed_X, y, batch_size=16, epochs=10, callbacks=callbacks_list, validation_split=0.3, shuffle=True, initial_epoch=6)

Train on 4454 samples, validate on 1909 samples
Epoch 7/10
4454/4454 [==============================] - 111s - loss: 1.0626 - acc: 0.4095 - val_loss: 1.0634 - val_acc: 0.3986
Epoch 8/10
4454/4454 [==============================] - 83s - loss: 1.0626 - acc: 0.4095 - val_loss: 1.0635 - val_acc: 0.3986
Epoch 9/10
4454/4454 [==============================] - 83s - loss: 1.0626 - acc: 0.4095 - val_loss: 1.0634 - val_acc: 0.3986
Epoch 10/10
4454/4454 [==============================] - 93s - loss: 1.0626 - acc: 0.4095 - val_loss: 1.0635 - val_acc: 0.3986


In [25]:
# serialize model to JSON
model_json = model.to_json()
with open("vgg16_top_only.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("vgg16_top_only.h5")
print("Saved model to disk")

Saved model to disk


In [26]:
printAllLayers()

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

# Fine-tuning more layers
See: https://www.cs.nyu.edu/~fergus/papers/zeilerECCV2014.pdf

Decided to fine-tune block 4 and 5 with top

In [28]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:11]:
   layer.trainable = False
for layer in model.layers[11:]:
   layer.trainable = True

In [29]:
printTrainableLayers()

11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [30]:
# serialize model to JSON
model_json = model.to_json()
with open("vgg16_block_4_5_before_training.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("vgg16_block_4_5_before_training.h5")
print("Saved model to disk")

Saved model to disk


In [31]:
# checkpoint
filepath="vgg16-block-4-5_-weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [32]:
tbCallBack = TensorBoard(log_dir='./Graph_4_5', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list.append(tbCallBack)